# FinGuard IntelliAgent - Milestone 5: Action Layer Demo

**Author:** Alfred Munga  
**Date:** January 2025  
**License:** MIT

---

## Overview

This notebook demonstrates **Milestone 5** of the FinGuard IntelliAgent ADK capstone project:

### 🎯 Key Concepts Implemented

1. **Task-Oriented Tool Design** (Agent Tools p.18)
   - Tools represent business tasks ("Get Unpaid Invoices", "Send Payment Request")
   - NOT raw API calls
   - Clear, actionable names for LLM tool selection

2. **Idempotency** (Prototype to Production p.21)
   - Payment requests check invoice status before execution
   - Prevents duplicate charges to customers
   - Status transitions: `overdue` → `processing` → `paid`

3. **Separation of Concerns**
   - Tools (business logic) vs. Services (external API integration)
   - `InvoiceDataManager` handles persistence
   - `DarajaService` handles M-Pesa API simulation

---

## What We'll Cover

1. **Scenario 1**: "Who owes me money?" (Retrieve unpaid invoices)
2. **Scenario 2**: "Send payment request to Client X" (Action execution)
3. **Scenario 3**: Try duplicate payment (Idempotency protection)
4. **Scenario 4**: Simulate payment completion and check status

---

## Architecture

```
┌────────────────┐
│  Agent/LLM     │  "Send payment to Rafiki Motors"
└────────┬───────┘
         │
         ▼
┌────────────────────────────┐
│  Tools (invoice_ops.py)    │
│  - GetUnpaidInvoicesTool   │  Business Logic Layer
│  - SendPaymentRequestTool  │  (Task-Oriented)
└────────┬───────────────────┘
         │
         ▼
┌─────────────────────────────┐
│  Services                   │
│  - InvoiceDataManager       │  External Integration Layer
│  - DarajaService (M-Pesa)   │  (Separation of Concerns)
└─────────────────────────────┘
```

---

Let's begin!

## 🚀 Setup

Import the tools and configure logging.

In [ ]:
import sys
import json
import logging
from pathlib import Path
from datetime import datetime

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import tools
from tools.invoice_ops import (
    GetUnpaidInvoicesTool,
    SendPaymentRequestTool,
    GetUnpaidInvoicesInput,
    SendPaymentRequestInput,
    InvoiceDataManager
)

from backend.services.daraja_service import DarajaService

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Setup complete!")
print(f"📂 Project root: {project_root}")

## 📋 Scenario 1: "Who owes me money?"

**Business Context:**  
You're a small business owner checking your accounts receivable.

**Tool Used:** `GetUnpaidInvoicesTool`

**Key Features:**
- Returns only **essential fields** (context optimization)
- Sorts by **days overdue** (most urgent first)
- Calculates **total outstanding** amount

In [ ]:
# Initialize the tool
get_invoices_tool = GetUnpaidInvoicesTool()

# Run the tool
result = get_invoices_tool.run(
    GetUnpaidInvoicesInput(user_id="demo_user")
)

# Display results
print("=" * 70)
print("📊 UNPAID INVOICES REPORT")
print("=" * 70)

if result['success']:
    print(f"\n✅ Found {result['total_count']} unpaid invoices")
    print(f"💰 Total Outstanding: {result['currency']} {result['total_amount']:,.2f}")
    
    if result['invoices']:
        print("\n📋 TOP 5 MOST OVERDUE:")
        print("-" * 70)
        
        for i, inv in enumerate(result['invoices'][:5], 1):
            print(f"\n{i}. {inv['invoice_id']} - {inv['customer_name']}")
            print(f"   Amount: {result['currency']} {inv['amount_outstanding']:,.2f}")
            print(f"   Due Date: {inv['due_date']}")
            print(f"   ⚠️  {inv['days_overdue']} days overdue")
            print(f"   📞 Contact: {inv['customer_phone']}")
else:
    print(f"❌ Error: {result.get('error')}")

print("\n" + "=" * 70)

## 💳 Scenario 2: "Send payment request to the most overdue client"

**Business Context:**  
You want to collect payment from your most overdue client using M-Pesa STK Push.

**Tool Used:** `SendPaymentRequestTool`

**Key Features:**
- ✅ **Idempotency check**: Won't send duplicate if already processing
- ✅ **Status validation**: Rejects paid invoices
- ✅ **Phone normalization**: Handles +254 and 254 formats
- ✅ **Atomic state update**: Invoice status → `processing`

In [ ]:
# Select the most overdue invoice
if result['success'] and result['invoices']:
    target_invoice = result['invoices'][0]  # Most overdue
    invoice_id = target_invoice['invoice_id']
    
    print("=" * 70)
    print("💳 SENDING PAYMENT REQUEST")
    print("=" * 70)
    print(f"\n📄 Invoice: {invoice_id}")
    print(f"👤 Customer: {target_invoice['customer_name']}")
    print(f"💰 Amount: KES {target_invoice['amount_outstanding']:,.2f}")
    print(f"⚠️  Days Overdue: {target_invoice['days_overdue']}")
    print("\n🚀 Initiating M-Pesa STK Push...\n")
    
    # Initialize tool
    send_payment_tool = SendPaymentRequestTool()
    
    # Send payment request
    payment_result = send_payment_tool.run(
        SendPaymentRequestInput(invoice_id=invoice_id)
    )
    
    # Display result
    if payment_result['success']:
        print("✅ SUCCESS! Payment request sent.")
        print(f"\n📱 Checkout Request ID: {payment_result['checkout_request_id']}")
        print(f"🏦 Merchant Request ID: {payment_result['merchant_request_id']}")
        print(f"📊 Status: {payment_result['status'].upper()}")
        print(f"\n💬 Message: {payment_result['message']}")
    else:
        print("❌ FAILED to send payment request.")
        print(f"\n⚠️  Error: {payment_result.get('error')}")
        if payment_result.get('message'):
            print(f"💬 Details: {payment_result['message']}")
    
    print("\n" + "=" * 70)
else:
    print("❌ No unpaid invoices found.")

## 🔒 Scenario 3: Try sending payment request AGAIN (Idempotency Test)

**Business Context:**  
What happens if we accidentally try to send the same payment request twice?

**Expected Behavior:**  
The tool should **reject** the duplicate request with a clear explanation.

**Why This Matters:**  
- Prevents **double-charging** customers
- Avoids **confusing** customers with multiple prompts
- Protects against **accidental** re-execution by agents

This is a core principle from **Prototype to Production (p.21)**.

In [ ]:
print("=" * 70)
print("🔒 IDEMPOTENCY TEST: Trying duplicate payment request")
print("=" * 70)
print(f"\n📄 Invoice: {invoice_id} (same as above)")
print("\n🚨 Attempting to send payment request AGAIN...\n")

# Try to send payment request again
duplicate_result = send_payment_tool.run(
    SendPaymentRequestInput(invoice_id=invoice_id)
)

# Display result
if not duplicate_result['success']:
    print("✅ IDEMPOTENCY CHECK WORKED!")
    print(f"\n🚫 Reason: {duplicate_result.get('reason', 'Unknown')}")
    print(f"📊 Current Status: {duplicate_result.get('status', 'Unknown')}")
    
    if duplicate_result.get('checkout_request_id'):
        print(f"🔗 Existing Checkout ID: {duplicate_result['checkout_request_id']}")
    
    if duplicate_result.get('message'):
        print(f"\n💬 Message:\n{duplicate_result['message']}")
else:
    print("❌ IDEMPOTENCY CHECK FAILED!")
    print("⚠️  Duplicate payment request was sent (this shouldn't happen!)")
    print(f"\n📱 New Checkout ID: {duplicate_result.get('checkout_request_id')}")

print("\n" + "=" * 70)

## ✅ Scenario 4: Simulate payment completion and verify status

**Business Context:**  
In production, M-Pesa would send a callback when the customer completes payment.  
For testing, we'll **simulate** payment completion.

**What We'll Do:**
1. Simulate customer entering PIN and completing payment
2. Check payment status via Daraja Service
3. (In production) Update invoice status to `paid`

In [ ]:
print("=" * 70)
print("✅ SIMULATING PAYMENT COMPLETION")
print("=" * 70)

# Get checkout request ID from previous payment
if payment_result.get('success'):
    checkout_id = payment_result['checkout_request_id']
    print(f"\n📱 Checkout Request ID: {checkout_id}")
    
    # Initialize Daraja service
    daraja = DarajaService()
    
    # Check initial status
    print("\n1️⃣ Checking initial status...")
    status_check_1 = daraja.get_payment_status(checkout_id)
    print(f"   Status: {status_check_1['status']}")
    
    # Simulate customer completing payment
    print("\n2️⃣ Simulating customer entering M-Pesa PIN and completing payment...")
    completion = daraja.simulate_payment_completion(checkout_id)
    print(f"   ✅ Payment simulated: {completion['message']}")
    
    # Check final status
    print("\n3️⃣ Checking final status...")
    status_check_2 = daraja.get_payment_status(checkout_id)
    print(f"   Status: {status_check_2['status']}")
    
    if status_check_2['status'] == 'COMPLETED':
        print("\n✅ Payment completed successfully!")
        print("\n📝 In production, this would trigger:")
        print("   - M-Pesa callback to your server")
        print("   - Invoice status update to 'paid'")
        print("   - Receipt generation")
        print("   - Customer notification (SMS/email)")
    else:
        print(f"\n⚠️  Payment status: {status_check_2['status']}")
else:
    print("\n❌ No payment to complete (previous request failed)")

print("\n" + "=" * 70)

## 📊 Summary: ADK Principles in Action

### 1. ✅ Task-Oriented Tool Design (Agent Tools p.18)

**What we did:**
- Created tools that represent **business tasks** ("Get Unpaid Invoices", "Send Payment Request")
- NOT raw API calls like `read_file()` or `http_post()`

**Why it matters:**
- Easier for LLMs to select the right tool
- Clear action semantics
- Better user experience ("Send payment to X" vs. "Call API endpoint Y")

---

### 2. ✅ Idempotency (Prototype to Production p.21)

**What we did:**
- `SendPaymentRequestTool` checks invoice status BEFORE sending M-Pesa request
- Rejects duplicates with clear explanation

**Why it matters:**
- Prevents **double-charging** customers (critical for payments!)
- Safe for agents to retry operations
- Meets production safety standards

**Test Result:** ✅ Duplicate request rejected successfully

---

### 3. ✅ Separation of Concerns

**Architecture:**
```
Tools Layer (invoice_ops.py)     → Business logic
Service Layer (daraja_service.py) → External API integration
Data Layer (invoices.json)        → Persistence
```

**Why it matters:**
- Easy to **swap** mock M-Pesa for production API (just change service)
- Easy to **test** tools independently
- Clean code structure

---

### 4. ✅ Context Optimization

**What we did:**
- `GetUnpaidInvoicesTool` returns only **essential fields** (ID, Customer, Amount, Due Date)
- Omits unnecessary data (invoice line items, metadata, etc.)

**Why it matters:**
- Saves **context tokens** for LLM processing
- Faster responses
- Lower costs

---

## 🎯 What's Next?

1. **Milestone 6**: Multi-Agent Coordination (if applicable)
2. **Production Deployment**:
   - Replace mock Daraja with real API
   - Add M-Pesa callback handler
   - Implement receipt generation
3. **Advanced Features**:
   - Automatic payment reminders
   - Payment plans
   - Multi-currency support

---

## 📚 References

- **Agent Development Kit (ADK) Whitepaper**: Agent Tools section (p.18)
- **Prototype to Production Guide**: Idempotency patterns (p.21)
- **Safaricom Daraja API**: [https://developer.safaricom.co.ke](https://developer.safaricom.co.ke)

---

**End of Milestone 5 Demo**